In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [55]:
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection.model_selection import _backtesting_forecaster

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

In [62]:
    
y_with_index = y.copy()
y_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')
exog_with_index = exog.copy()
exog_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')
forecaster = ForecasterAutoreg(regressor=Ridge(random_state=123), 
                                   lags=3, binner_kwargs={'n_bins': 15})


metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster              = forecaster,
                                    y                       = y_with_index,
                                    exog                    = exog_with_index,
                                    refit                   = 3,
                                    initial_train_size      = len(y_with_index) - 20,
                                    fixed_train_size        = True,
                                    gap                     = 3,
                                    allow_incomplete_fold   = False,
                                    steps                   = 4,
                                    metric                  = 'mean_squared_error',
                                    interval                = [5, 95],
                                    n_boot                  = 500,
                                    random_state            = 123,
                                    use_in_sample_residuals = True,
                                    verbose                 = False,
                                    n_jobs                  = 2
                                )

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\model_selection.py:406: IgnoredArgumentWarning: If `refit` is an integer other than 1 (intermittent refit). `n_jobs` is set to 1 to avoid unexpected results during parallelization. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=IgnoredArgumentWarning)
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?it/s]

In [63]:
print(metric)

   mean_squared_error
0            0.060992


In [64]:
backtest_predictions.to_numpy()

array([[0.51878642, 0.19491548, 0.86522758],
       [0.49269791, 0.15037971, 0.85925162],
       [0.49380441, 0.1639645 , 0.89759979],
       [0.51467463, 0.15567332, 0.88191791],
       [0.52690045, 0.20302951, 0.87334162],
       [0.51731996, 0.17500175, 0.88387367],
       [0.51290311, 0.18306321, 0.9166985 ],
       [0.50334306, 0.14434174, 0.87058633],
       [0.50171526, 0.17784432, 0.84815642],
       [0.50946908, 0.16715088, 0.87602279],
       [0.50200357, 0.17216366, 0.90579895],
       [0.50436041, 0.1453591 , 0.87160369],
       [0.4534189 , 0.08003548, 0.82515615],
       [0.52621695, 0.17498193, 0.91058562],
       [0.50477802, 0.14697488, 0.88219215],
       [0.52235258, 0.14985376, 0.90260343]])

In [53]:
backtest_predictions.index

DatetimeIndex(['2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06',
               '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14',
               '2022-02-15', '2022-02-16', '2022-02-17'],
              dtype='datetime64[ns]', freq='D')